In [63]:
import pandas as pd
from pathlib import Path

In [64]:
file_path = 'gephi_edges.csv'

In [65]:
all_ratings = pd.read_csv(file_path, header=0)

In [66]:
all_ratings['Favorable'] = ~all_ratings['target'].isnull()
#print(all_ratings['source'].max())
ratings = all_ratings[all_ratings['source'].isin(range(396))]

In [67]:
favorable_ratings = all_ratings[all_ratings['Favorable']]

In [68]:
favorable_reviews_by_users = dict((k, frozenset(v.values)) for k, v in favorable_ratings.groupby('source')['target'])

In [69]:
num_favorable_by_story = all_ratings[['target', 'Favorable']].groupby('target').sum()

In [70]:
num_favorable_by_story.sort_values('Favorable', ascending=False).head()

,Favorable
target,
4.0,181
34.0,154
21.0,130
22.0,123
36.0,119


In [71]:
frequent_itemsets = dict()
min_support = 50
frequent_itemsets[1] = dict((frozenset((target,)), row['Favorable']) for target, row in num_favorable_by_story.iterrows() if row['Favorable'] > min_support)

In [72]:
from collections import defaultdict

def find_frequent_itemsets(favorable_reviews_by_users, k_1_itemsets, min_support):
    counts = defaultdict(int)
    for user, reviews in favorable_reviews_by_users.items():
        for itemset in k_1_itemsets:
            if itemset.issubset(reviews):
                for other_reviewed_movie in reviews - itemset:
                    current_superset = itemset | frozenset((other_reviewed_movie,))
                    counts[current_superset] += 1
    return dict([(itemset, frequency) for itemset, frequency in counts.items() if frequency >= min_support])

In [73]:
import sys
for k in range(2, 20):
    cur_frequent_itemsets = find_frequent_itemsets(favorable_reviews_by_users, frequent_itemsets[k-1], min_support)
    frequent_itemsets[k] = cur_frequent_itemsets
    if len(cur_frequent_itemsets) == 0:
        print(f"Did not find any frequent itemsets of length {k}.")
        sys.stdout.flush()
        break
    else:
        print(f"I found {len(cur_frequent_itemsets)} frequent itemsets of length {k}.")
        sys.stdout.flush()
        frequent_itemsets[k] = cur_frequent_itemsets

I found 413 frequent itemsets of length 2.
I found 2208 frequent itemsets of length 3.
I found 6426 frequent itemsets of length 4.
I found 11402 frequent itemsets of length 5.
I found 12617 frequent itemsets of length 6.
I found 8799 frequent itemsets of length 7.
I found 3709 frequent itemsets of length 8.
I found 844 frequent itemsets of length 9.
I found 85 frequent itemsets of length 10.
I found 2 frequent itemsets of length 11.
Did not find any frequent itemsets of length 12.


In [74]:
candidate_rules = []
for itemset_length, itemset_counts in frequent_itemsets.items():
    for itemset in itemset_counts.keys():
        for conclusion in itemset:
            premise = itemset - set((conclusion,))
            candidate_rules.append((premise, conclusion))

In [75]:
correct_counts = defaultdict()
incorrect_counts =  defaultdict()

correct_counts = defaultdict(int)
incorrect_counts = defaultdict(int)
for user, reviews in favorable_reviews_by_users.items():
    for candidate_rule in candidate_rules:
        premise, conclusion = candidate_rule
        if premise.issubset(reviews):
            if conclusion in reviews:
                correct_counts[candidate_rule] += 1
            else:
                incorrect_counts[candidate_rule] += 1

In [76]:
rule_confidence = {candidate_rule: correct_counts[candidate_rule]/float
(correct_counts[candidate_rule] + incorrect_counts[candidate_rule]) for 
candidate_rule in candidate_rules} 

In [77]:
from operator import itemgetter
sorted_confidence = sorted(rule_confidence.items(), key=itemgetter(1), 
reverse=True)

for index in range(5):
    print(f"Rule #{index+1}")
    (premise, conclusion) = sorted_confidence[index][0]
    print(f"Rule: If a person recommends {premise} they will also recommend {conclusion}.")
    print(f"- Confidence: {rule_confidence[(premise, conclusion)]}")
    print("")

Rule #1
Rule: If a person recommends frozenset({102.0}) they will also recommend 4.0.
- Confidence: 1.0

Rule #2
Rule: If a person recommends frozenset({115.0, 78.0}) they will also recommend 27.0.
- Confidence: 1.0

Rule #3
Rule: If a person recommends frozenset({27.0, 94.0}) they will also recommend 115.0.
- Confidence: 1.0

Rule #4
Rule: If a person recommends frozenset({34.0, 115.0}) they will also recommend 27.0.
- Confidence: 1.0

Rule #5
Rule: If a person recommends frozenset({74.0, 108.0}) they will also recommend 101.0.
- Confidence: 1.0



In [86]:
from all_names import all_author_and_student_names

In [87]:
names = all_author_and_student_names
print(names)

{1: ['Lovecraft', 'Supernatural Horror in Fiction'], 2: ['Lois Tyson', 'Lois', 'Tyson', 'Psychoanalytic Criticism', 'Critical Theory Today', 'Critical Theory Today: A User-Friendly Guide'], 3: ['Michael Moorcock', 'Moorcock', 'Foreword'], 4: ['Jeff', 'VanderMeer', 'Ann', 'VanderMeer', 'Introduction'], 5: ['Sigmund Freud', 'Sigmund', 'Freud', 'The Uncanny'], 6: ['Alfred Kubin', 'Alfred', 'Kubin', 'The Other Side'], 7: ['F. Marion Crawford', 'Marion Crawford', 'The Screaming Skull'], 8: ['Algernon Blackwood', 'Algernon', 'Blackwood', 'The Willows'], 9: ['Saki', 'Sredni Vashtar'], 10: ['M. R. James', 'M. R.', 'Casting the Runes'], 11: ['Lord Dunsany', 'How Nuth Would Have Practised His Art Upon the Gnoles'], 12: ['Gustav Meyrink', 'Gustav', 'Meyrink', 'The Man in the Bottle'], 13: ['Georg Heym', 'Heym', 'The Dissection'], 14: ['Hans Heinz Ewers', 'Hans', 'Heinz', 'Ewers', 'The Spider'], 15: ['Rabindranath Tagore', 'Rabindranath', 'Tagore', 'The Hungry Stones'], 16: ['Luigi Ugolini', 'Luig

In [83]:
k_v_exchanged = {}

for key, value in names.items():
    if value not in k_v_exchanged:
        k_v_exchanged[value] = [key]
    else:
        k_v_exchanged[value].append(key)

TypeError: unhashable type: 'list'